In [25]:
import copy
import heapq
from typing import Dict, List, Set, Tuple

In [26]:
class Sudoku:
    
    full_str_set = {'.','1','2','3','4','5','6','7','8','9'}
    str_set = {'1','2','3','4','5','6','7','8','9'}

    @staticmethod
    def _is_solved(board: List[List[str]]) -> bool:
        '''Returns boolean if board is solved'''
        if board is None: return False

        temp = copy.deepcopy(board)

        r, c, s, e = Sudoku.generate_rcs_sets(temp)

        if len(r)==9 and len(c)==9 and len(s)==9 and len(e)==0: 
            return True
        else:
            return False

    @staticmethod
    def check_board_validity(board: List[List[str]])->bool:
        '''Test if input board is valid.'''
        if not isinstance(board, list) or len(board)!=9: return False

        for row in board:
            if not isinstance(row,list) or len(row)!=9: return False
            for value in row:
                if not isinstance(value,str) or value not in Sudoku.full_str_set: return False

        return True

    @staticmethod
    def generate_rcs_sets(board: List[List[str]])-> Tuple[Dict[int,Set[int]], Dict[int,Set[int]], Dict[int,Set[int]], Set[Tuple[int]]]:
        '''Generate board representation of rows, cols, squares, and empties.'''
        temp = copy.deepcopy(board)
        rows = {i: set() for i in range(0, 9)}
        cols = {i: set() for i in range(0, 9)}
        squares = {i: set() for i in range(0, 9)}
        empties = set()
        for i in range(9):
            for j in range(9):
                n = temp[i][j]
                k = 3*(i//3)+j//3
                if n == '.':
                    empties.add((i,j,k))
                else:
                    rows[i].add(n)
                    cols[j].add(n)
                    squares[k].add(n)

        return rows,cols,squares,empties
    
    
    def __init__(self, unsolved: List[List[str]]):
        if not self.check_board_validity(unsolved):
            raise ValueError("Board is invalid.")
        self.unsolved_board = unsolved 
        self.solved_board = None
        self.is_solved = False
        

    def solve_board(self) -> None | List[List[str]]:
        '''Takes unsolved_board attribute and generates solved_board and is_solved attributes. Also returns board is solved or None if not.'''
        temp = copy.deepcopy(self.unsolved_board)
        
        rows, cols, squares, empties = self.generate_rcs_sets(temp)

        heap = []
        count = 0
        for i, j, k in empties:
            possibilities = Sudoku.str_set - (rows[i] | cols[j] | squares[k])
            if len(possibilities) == 1:
                n = possibilities.pop()
                rows[i].add(n)
                cols[j].add(n)
                squares[k].add(n)
                temp[i][j] = n
            else:
                heapq.heappush(heap, (count, len(possibilities), i, j, k))

        solution = []
        count += 1
        bifurcationLength = 1
        while heap:
            itemCount, _, i, j, k = heapq.heappop(heap)

            if itemCount > count:
                count = itemCount
                bifurcationLength += 1
                print(f'Incrementing bifurcation length to {bifurcationLength}')

            if bifurcationLength > 9:
                return None

            if itemCount < count:
                heapq.heappush(heap,(count, len(possibilities), i, j, k))
                continue

            possibilities = Sudoku.str_set - (rows[i] | cols[j] | squares[k])
            
            while not possibilities:
                if not solution:
                    print('Failed to find solution due to empty solution stack')
                    return None
                # backtrack adding attempted squares back minus n
                heapq.heappush(heap, (count + 1, len(possibilities), i, j, k))
                # backtrack until possibilities are not empty
                i, j, k, n, possibilities = solution.pop()
                rows[i].remove(n)
                cols[j].remove(n)
                squares[k].remove(n)

            if len(possibilities) <= bifurcationLength:
                n = possibilities.pop()
                rows[i].add(n)
                cols[j].add(n)
                squares[k].add(n)
                solution.append((i,j,k,n,possibilities))
                bifurcationLength = 1
                continue

            heapq.heappush(heap, (count + 1, len(possibilities), i, j, k))
            

        while solution:
            i, j, k, n, possibilities = solution.pop()
            temp[i][j] = n

        if self._is_solved(temp):
            self.is_solved=True
            self.solved_board = temp
            return temp
        else:
            self.is_solved = False
            self.solved_board = None
            return None
            
            
    
            

In [27]:
unsolved = [["5","3",".",".","7",".",".",".","."],["6",".",".","1","9","5",".",".","."],[".","9","8",".",".",".",".","6","."],["8",".",".",".","6",".",".",".","3"],["4",".",".","8",".","3",".",".","1"],["7",".",".",".","2",".",".",".","6"],[".","6",".",".",".",".","2","8","."],[".",".",".","4","1","9",".",".","5"],[".",".",".",".","8",".",".","7","9"]]
solved = [["5","3","4","6","7","8","9","1","2"],["6","7","2","1","9","5","3","4","8"],["1","9","8","3","4","2","5","6","7"],["8","5","9","7","6","1","4","2","3"],["4","2","6","8","5","3","7","9","1"],["7","1","3","9","2","4","8","5","6"],["9","6","1","5","3","7","2","8","4"],["2","8","7","4","1","9","6","3","5"],["3","4","5","2","8","6","1","7","9"]]

sudoku0 = Sudoku(unsolved)

# test class __init__
# print(sudoku0.is_solved)
# print(sudoku0.unsolved_board)
# print(sudoku0.solved_board)

# test class validity method
# print(Sudoku.check_board_validity(unsolved))
# print(Sudoku.check_board_validity(solved))

# test class generate rcs sets method
# print(Sudoku.generate_rcs_sets(unsolved))
# print(Sudoku.generate_rcs_sets(solved))

# test class solve board method
sudoku0.solve_board()
print(sudoku0.solved_board)
print(sudoku0.is_solved)

Incrementing bifurcation length to 2
Incrementing bifurcation length to 2
Incrementing bifurcation length to 2
Incrementing bifurcation length to 2
[['5', '3', '4', '6', '7', '8', '9', '1', '2'], ['6', '7', '2', '1', '9', '5', '3', '4', '8'], ['1', '9', '8', '3', '4', '2', '5', '6', '7'], ['8', '5', '9', '7', '6', '1', '4', '2', '3'], ['4', '2', '6', '8', '5', '3', '7', '9', '1'], ['7', '1', '3', '9', '2', '4', '8', '5', '6'], ['9', '6', '1', '5', '3', '7', '2', '8', '4'], ['2', '8', '7', '4', '1', '9', '6', '3', '5'], ['3', '4', '5', '2', '8', '6', '1', '7', '9']]
True
